# 単語と文章のベクトル化

自然言語のままで各種分析を行う際は、単語と文章のベクトル化が必要になります。

単語のベクトル化：word2vec

文章のベクトル化：doc2vec


# 学習済みモデル

基本的に、word2vecは、著名な学習済みデータセットを使用することが多いです。

doc2vecは、案件ごとに学習することが多いです。

http://blog.hassaku-labs.com/post/pretrained-word2vec/

＊学習済みモデルの「entity_vector.model.bin」は、下記URLよりダウンロードしておいてください。

http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/

# word2vec（既存モデルの使用）

In [11]:
# 読み込み
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('./data/entity_vector.model.bin', binary=True)

In [12]:
# 単語のベクトル化
model.wv["イチロー"]

c:\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


array([ 0.09032071, -0.05341686,  0.28220174,  0.29330444,  0.05328318,
        0.14241028, -0.24687128,  0.18441112,  0.02650695,  0.0828563 ,
       -0.426362  , -0.02014246,  0.02603034, -0.03726895,  0.19294073,
       -0.12511627,  0.5407648 ,  0.05329642, -0.114064  , -0.08794833,
        0.48768535,  0.16046745,  0.0785282 , -0.49404824,  0.11478704,
       -0.32564625, -0.55675274, -0.30036446, -0.20920439,  0.06128695,
       -0.05599485, -0.08785352, -0.17117216,  0.28584853,  0.01225359,
        0.15091456, -0.07321789, -0.07828152,  0.16382505,  0.38058066,
        0.19476666,  0.06204469, -0.27429718,  0.04294476, -0.05257293,
        0.05322105, -0.15351981, -0.6151462 ,  0.16540758,  0.36411384,
        0.03025797, -0.31749636,  0.2605783 , -0.05329959,  0.0940825 ,
       -0.1286651 , -0.19278173, -0.3219691 ,  0.10385962,  0.25326288,
       -0.00975231, -0.09564543, -0.3629564 , -0.20086282, -0.12936008,
        0.43610513, -0.36885667,  0.295815  , -0.12997843,  0.43

In [13]:
# 類似単語の抽出
results = model.most_similar(u"[指原莉乃]")
for result in results:
    print(result)

('[前田敦子]', 0.8524860143661499)
('[松井玲奈]', 0.8497596979141235)
('[柏木由紀]', 0.8481603860855103)
('[大島優子]', 0.8426693677902222)
('[渡辺麻友]', 0.836478590965271)
('[高橋みなみ]', 0.8302541971206665)
('[山本彩]', 0.8275360465049744)
('[HKT48]', 0.8238900899887085)
('[小嶋陽菜]', 0.8218119144439697)
('[島崎遥香]', 0.8172259330749512)


In [14]:
# 減算
results = model.most_similar(positive=[u'[妻]'],negative=[u'[愛人]'])
for result in results:
    print(result)

('ドレナージ', 0.4027651846408844)
('[分娩]', 0.3961508870124817)
('[座位]', 0.37961751222610474)
('[出産]', 0.37935060262680054)
('[生殖]', 0.3761950731277466)
('望ましい', 0.37324926257133484)
('[子育て]', 0.372659832239151)
('[歯科医師]', 0.3716592788696289)
('[歯学者]', 0.36931663751602173)
('[公認心理師]', 0.36690229177474976)


In [15]:
# 加算
results = model.most_similar(positive=[u'[妖精]',u'[悪]'])
for result in results:
    print(result)

('[精霊]', 0.8402377367019653)
('[魔女]', 0.8217898607254028)
('[悪魔]', 0.8189244270324707)
('悪霊', 0.8049401044845581)
('創造主', 0.7825872898101807)
('[吸血鬼]', 0.7805193662643433)
('[悪霊]', 0.779639482498169)
('精霊', 0.7730634212493896)
('魔物', 0.771123468875885)
('[エルフ]', 0.7656500339508057)


In [16]:
# 加算減算
results = model.most_similar(positive=[u'[イチロー]',u'[サッカー]'],negative=[u'[野球]'])
for result in results:
    print(result)

('[ロナウド]', 0.6781752109527588)
('[ジネディーヌ・ジダン]', 0.6686528325080872)
('[中田英寿]', 0.6643422842025757)
('[ジーコ]', 0.6631736755371094)
('[ロベルト・バッジョ]', 0.6589246988296509)
('[ロナウジーニョ]', 0.6574504971504211)
('[ディエゴ・マラドーナ]', 0.6535028219223022)
('ジダン', 0.6474759578704834)
('[デビッド・ベッカム]', 0.6473382115364075)
('[三浦知良]', 0.6451823711395264)


# doc2vec（自己学習）

使用するデータは、青空文庫にある小説です。

https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip

In [21]:
# データの読み込み
with open('./data/sanshiro.txt', encoding='sjis') as file:
    text = file.read()

# ファイル整形
import re
text = re.split('\-{5,}',text)[2]
text = re.split('底本：',text)[0]
text = text.replace('|', '')
text = re.sub('《.+?》', '', text)
text = re.sub('［＃.+?］', '',text)
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)
text = re.sub('　', '', text)
text = re.sub(' ', '', text)

print(text[:100])

print(len(text))


一
うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬいだ
171110


In [23]:
# 下記のようなリストに変換する
# [
#  [単語1,単語2,....]
#  [単語1,単語2,....]
#  [単語1,単語2,....]
# ]

# ライブラリの読み込み
from janome.tokenizer import Tokenizer
def extract_data(text):
    '''文章を渡したら、動詞と名詞のリストを返す
    引数：
    　text：一文
    返り値：
    　data：動詞と名詞のリスト
    処理概要：
    　1つの文章を受け取り、その中に含まれている名詞と動詞のリストを返す
    '''
    # インスタンス化
    t = Tokenizer()
    # 動詞と名詞のリスト
    data=[]
    # 形態素解析
    res = t.tokenize(text)
    for i in res:
        data.append(i.surface)
    return data

# テキストを「。」で区切る
text_list = text.split('。')

# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_data(text) for text in text_list]

In [24]:
# 形式の確認
print(len(word_list))

print(word_list[0])

5904
['一', '\n', 'うとうと', 'と', 'し', 'て', '目', 'が', 'さめる', 'と', '女', 'は', 'いつのまにか', '、', '隣', 'の', 'じいさん', 'と', '話', 'を', '始め', 'て', 'いる']


In [25]:
# ライブラリ読み込み
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [26]:
# 学習用データに変換
training_docs = []

for i in word_list:
    training_docs.append(
        TaggedDocument(words=i,tags=[str(i)])
    )

In [27]:
# モデルの学習
model = Doc2Vec(documents=training_docs, dm = 1, size=30, window=8, min_count=1, workers=4)

In [29]:
# 文章のベクトル化
model.wv[word_list[0]],model.wv[word_list[0]].shape

(array([[-1.19091141e+00, -7.98784614e-01, -1.08580160e+00,
         -7.86014855e-01,  8.11349332e-01, -7.95187116e-01,
          1.51691806e+00, -1.77765715e+00, -4.65694129e-01,
          2.31543827e+00, -2.23256886e-01, -2.00391245e+00,
          9.84670967e-02,  3.19117680e-02, -1.60752189e+00,
         -1.94346404e+00, -1.58391857e+00,  1.64841223e+00,
         -1.67279387e+00, -1.41362178e+00, -1.49163628e+00,
          9.50311184e-01, -3.27118337e-01,  9.72959459e-01,
          1.17340267e+00,  1.55865654e-01, -1.15609550e+00,
         -2.14889550e+00, -1.62270463e+00,  1.17871177e+00],
        [-6.08448088e-01,  1.36126018e+00, -1.40350032e+00,
         -5.35801232e-01,  1.14420462e+00,  9.97487307e-01,
          1.31184554e+00,  1.54259712e-01,  2.16970658e+00,
          1.58904159e+00,  1.01510954e+00, -2.66622043e+00,
         -1.26174271e+00, -1.62116277e+00, -1.80827916e+00,
         -3.24738717e+00, -4.01602447e-01, -1.21021681e-01,
         -3.89945722e+00,  6.38427883e-

In [31]:
# 試しに類似文書抽出
for i in range(3):
    print('==========================')
    print("文書A＝",''.join(word_list[i]))
    print()
    print("文書B＝",model.docvecs.most_similar(i)[0][0].replace('"','').replace('[','').replace(']','').replace("'","").replace(',','').replace(' ',''))
    print()

文書A＝ 一
うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている

文書B＝ 文芸は技術でもない、事務でもない

文書A＝ このじいさんはたしかに前の前の駅から乗ったいなか者である

文書B＝ 三四郎はしばらく我慢していたが、とうとう焦れったくなって、与次郎に、美禰子に関するすべての事実を隠さずに話してくれと請求した

文書A＝ 発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬいだと思ったら背中にお灸のあとがいっぱいあったので、三四郎の記憶に残っている

文書B＝ やがてまた動く気になったので腰を上げて、立ちながら靴の踵を向け直すと、丘の上りぎわの、薄く色づいた紅葉の間に、さっきの女の影が見えた



In [32]:
print("文書A＝",''.join(word_list[0]))
print()
print("文書B＝",''.join(word_list[100]))
print()
print("類似度",model.docvecs.similarity(0,100))

文書A＝ 一
うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている

文書B＝ うしろから女がついて来る

類似度 -0.12957396


# 実際の使い方

word2vecもdoc2vecも

NNなどの各種機械学習アルゴリズムで、自然言語を利用しようと思うと、自然言語の数値化は必須です。

その為、自然言語解析のベースとなるアルゴリズムです。

似た意味を持つ単語や「単語の意味」の加算減算することにより、類似した単語や文章の抽出が可能です。

実務では、質問投稿サイトなどで、似通った質問のレコメンドシステムや検索エンジンの構築が出来ます。